# Assembling our quarterback dataset
Along with the EDA, we also do the data wrangling to put together the dataset we'll use for our quarterback model.<br>

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import nfl_data_py as nfl
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [2]:
import re

In [3]:
day = get_current_weekday()

In [4]:
date_string = get_next_sunday(day)

In [5]:
week = calculate_nfl_week(date_string)

In [6]:
season = get_current_year()

In [7]:
import sqlite3

In [8]:
# Connect to the SQLite database
#These are the FD and DK player lists for the current week
conn = sqlite3.connect('nfl_dfs.db')

query_fd = "SELECT * FROM fd_table_" + str(week) + "_" + str(season)[2:]
query_dk = "SELECT * FROM dk_table_" + str(week) + "_" + str(season)[2:]

fanduel_df = pd.read_sql_query(query_fd, conn)
draftkings_df = pd.read_sql_query(query_dk, conn)

# Close the database connection
conn.close()

In [9]:
#The weekly and play-by-play data through the previous week
#We went into R Studio to get these and save them as CSVs that we can use here
weekly_df = pd.read_csv('weekly_data_' + str(season) + '_' + str(week) + '.csv')
pbp_df = pd.read_csv('pbp_data_' + str(season) + '_' + str(week) + '.csv', low_memory = False)

In [10]:
weekly_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,1,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,10.38,10.38
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,2,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,9.40,9.40
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,3,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,14.74,14.74
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,25.40,25.40
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,19.74,19.74


In [11]:
weekly_df.tail()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
13052,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,1,REG,...,0,-0.088616,0,-1.250000,0.032258,-0.020942,0.033727,0,1.8,2.8
13053,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,2,REG,...,0,0.238474,0,-1.166667,0.047619,-0.028037,0.051802,0,1.7,2.7
13054,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,3,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.8,0.8
13055,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,5.0,5.0
13056,00-0039921,T.Benson,Trey Benson,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,ARI,2024,6,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,2.6,2.6


In [12]:
pbp_df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.443521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,1.468819,NaN,NaN,NaN,NaN,NaN,0.440373,-44.037291
3,89,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.492192,0.727261,6.988125,6.0,0.60693,0.227598,0.389904,61.009598
4,115,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.325931,NaN,NaN,NaN,NaN,NaN,0.443575,-44.357494


In [13]:
pbp_df.tail()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
115007,4054,2024_06_WAS_BAL,2024101301,BAL,WAS,REG,6,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115008,4042,2024_06_WAS_BAL,2024101301,BAL,WAS,REG,6,BAL,home,WAS,...,1,1,0.512203,NaN,NaN,NaN,NaN,NaN,0.080861,-8.086147
115009,4078,2024_06_WAS_BAL,2024101301,BAL,WAS,REG,6,BAL,home,WAS,...,0,1,-0.475150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115010,4100,2024_06_WAS_BAL,2024101301,BAL,WAS,REG,6,BAL,home,WAS,...,0,1,-1.668953,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115011,4122,2024_06_WAS_BAL,2024101301,BAL,WAS,REG,6,BAL,home,WAS,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
qb_cols = ['player_id', 'player_name', 'player_display_name', 'position', 'recent_team', 'season', 'week',\
 'season_type', 'opponent_team', 'completions', 'attempts',\
 'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',\
 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',\
 'passing_yards_after_catch', 'passing_epa',\
 'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',\
 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_2pt_conversions',\
 'receptions', 'receiving_yards', 'receiving_tds',\
 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'racr', 'fantasy_points', 'fantasy_points_ppr']

# Weekly data
This weekly data will enable us to derive the fantasy points of each QB in every game.

In [15]:
weekly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13057 entries, 0 to 13056
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    13057 non-null  object 
 1   player_name                  13057 non-null  object 
 2   player_display_name          13057 non-null  object 
 3   position                     13057 non-null  object 
 4   position_group               13057 non-null  object 
 5   headshot_url                 12981 non-null  object 
 6   recent_team                  13057 non-null  object 
 7   season                       13057 non-null  int64  
 8   week                         13057 non-null  int64  
 9   season_type                  13057 non-null  object 
 10  opponent_team                13057 non-null  object 
 11  completions                  13057 non-null  int64  
 12  attempts                     13057 non-null  int64  
 13  passing_yards   

In [16]:
pbp_df = pbp_df.replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [17]:
weekly_df = weekly_df.replace({'LA' : 'LAR', 'JAC' : 'JAX'})

In [18]:
qb_df = weekly_df[weekly_df['position'] == 'QB']

In [19]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1609 entries, 0 to 13044
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    1609 non-null   object 
 1   player_name                  1609 non-null   object 
 2   player_display_name          1609 non-null   object 
 3   position                     1609 non-null   object 
 4   position_group               1609 non-null   object 
 5   headshot_url                 1601 non-null   object 
 6   recent_team                  1609 non-null   object 
 7   season                       1609 non-null   int64  
 8   week                         1609 non-null   int64  
 9   season_type                  1609 non-null   object 
 10  opponent_team                1609 non-null   object 
 11  completions                  1609 non-null   int64  
 12  attempts                     1609 non-null   int64  
 13  passing_yards         

In [20]:
qb_df = qb_df[qb_cols]

In [21]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1609 entries, 0 to 13044
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player_id                  1609 non-null   object 
 1   player_name                1609 non-null   object 
 2   player_display_name        1609 non-null   object 
 3   position                   1609 non-null   object 
 4   recent_team                1609 non-null   object 
 5   season                     1609 non-null   int64  
 6   week                       1609 non-null   int64  
 7   season_type                1609 non-null   object 
 8   opponent_team              1609 non-null   object 
 9   completions                1609 non-null   int64  
 10  attempts                   1609 non-null   int64  
 11  passing_yards              1609 non-null   int64  
 12  passing_tds                1609 non-null   int64  
 13  interceptions              1609 non-null   int64  
 

In [22]:
fanduel_df = fanduel_df[fanduel_df['position'] == 'QB']
draftkings_df = draftkings_df[draftkings_df['position'] == 'QB']

# Setting up for concatenation
We need to add the current week's player list to the data that goes through last week so we can derive L8 variables for the current week. There are no DraftKings names that aren't in the FanDuel list, so we can just use the FanDuel list to concat.

In [23]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week
2,108151-62239,Josh Allen,QB,9200,BUF,TEN,1,Active,10-20-2024,7
3,108151-102785,Jayden Daniels,QB,9100,WAS,CAR,1,Active,10-20-2024,7
8,108151-69017,Jordan Love,QB,8700,GB,HOU,1,Active,10-20-2024,7
11,108151-69531,Jalen Hurts,QB,8500,PHI,NYG,0,Active,10-20-2024,7
13,108151-129471,CJ Stroud,QB,8400,HOU,GB,0,Active,10-20-2024,7


In [24]:
def clean_name(name):
    # Remove periods between initials like C.J., D.J. (case-sensitive)
    name = re.sub(r'\b([A-Z])\.\s*([A-Z])\.\b', r'\1\2', name)
    
    # Remove common suffixes like Jr., Sr., III, II, IV (case-sensitive)
    cleaned_name = re.sub(r'(\,|\.|Sr|Jr|III|II|IV)', '', name).strip()
    
    return cleaned_name

In [25]:
qb_df['player_display_name'] = qb_df['player_display_name'].apply(clean_name)

In [26]:
from rapidfuzz import process, fuzz

In [27]:
SIMILARITY_THRESHOLD = 80

In [28]:
def fuzzy_match(name, dk_names):
    match, score, _ = process.extractOne(name, dk_names, scorer=fuzz.token_sort_ratio)
    return match if score >= SIMILARITY_THRESHOLD else None

In [29]:
qb_names = qb_df['player_display_name']

In [30]:
fanduel_df['matched_name'] = fanduel_df['name'].apply(lambda x: fuzzy_match(x, qb_names))

In [31]:
unmatched_in_fd = fanduel_df[fanduel_df['matched_name'].isna()]

In [32]:
#If all unmatched players have a salary of 6000, the QB minimum, then we don't need to worry about it
unmatched_in_fd[unmatched_in_fd['salary'] > 6000]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [33]:
fanduel_df[(fanduel_df['name'] != fanduel_df['matched_name']) & (fanduel_df['matched_name'].notna())]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [34]:
draftkings_df['matched_name'] = draftkings_df['name'].apply(lambda x: fuzzy_match(x, qb_names))

In [35]:
unmatched_in_dk = draftkings_df[draftkings_df['matched_name'].isna()]

In [36]:
unmatched_in_dk[unmatched_in_dk['salary'] > 4000]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
89,36290742,Michael Penix,QB,5300,ATL,SEA,1,Active,10-20-2024,7,None


In [37]:
draftkings_df[(draftkings_df['name'] != draftkings_df['matched_name']) & (draftkings_df['matched_name'].notna())]

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name


In [38]:
# fanduel_df.replace({'Gardner Minshew II': 'Gardner Minshew', 'PJ Walker': 'P.J. Walker'}, inplace = True)
# draftkings_df.replace({'Gardner Minshew II': 'Gardner Minshew', 'PJ Walker': 'P.J. Walker'}, inplace = True)

In [39]:
new_df = fanduel_df[['name', 'position', 'team', 'opponent', 'week']]
new_df.rename(columns = {'name': 'player_display_name', 'team' : 'recent_team', 'opponent': 'opponent_team'}, inplace = True)
new_df['season'] = season

C:\Users\Owner\AppData\Local\Temp\ipykernel_35456\3609715700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns = {'name': 'player_display_name', 'team' : 'recent_team', 'opponent': 'opponent_team'}, inplace = True)
C:\Users\Owner\AppData\Local\Temp\ipykernel_35456\3609715700.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['season'] = season


In [40]:
qb_df.drop(columns = ['player_id', 'player_name', 'season_type'], inplace = True)

In [41]:
qb_df = pd.concat([qb_df, new_df], axis = 0)

In [42]:
qb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1706 entries, 0 to 168
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   player_display_name        1706 non-null   object 
 1   position                   1706 non-null   object 
 2   recent_team                1706 non-null   object 
 3   season                     1706 non-null   int64  
 4   week                       1706 non-null   int64  
 5   opponent_team              1706 non-null   object 
 6   completions                1609 non-null   float64
 7   attempts                   1609 non-null   float64
 8   passing_yards              1609 non-null   float64
 9   passing_tds                1609 non-null   float64
 10  interceptions              1609 non-null   float64
 11  sacks                      1609 non-null   float64
 12  sack_yards                 1609 non-null   float64
 13  sack_fumbles               1609 non-null   float64
 14

In [43]:
qb_cols_to_group = ['season', 'week', 'player_display_name', 'recent_team', 'opponent_team']

In [44]:
qb_scoring_cols = ['passing_yards', 'passing_tds', 'interceptions', 'passing_2pt_conversions', 'rushing_yards',\
                   'rushing_tds', 'rushing_fumbles_lost', 'rushing_2pt_conversions', 'receptions', 'receiving_yards',\
                   'receiving_tds', 'receiving_fumbles_lost', 'receiving_2pt_conversions', 'sack_fumbles_lost' ]

In [45]:
qb_df = qb_df.set_index(qb_cols_to_group, drop = True)

# Fantasy points
This is where we calculate FanDuel and DraftKings points.

In [46]:
qb_df['FD_Pts'] = (qb_df['passing_yards'] * 0.04) + (qb_df['rushing_tds'] * 6) + (qb_df['rushing_yards'] * 0.1)\
+ (qb_df['passing_tds'] * 4) + (qb_df['receiving_yards'] * 0.1) + (qb_df['receiving_tds'] * 6) + (qb_df['receptions'] * 0.5)\
+ (qb_df['rushing_2pt_conversions'] * 2) + (qb_df['passing_2pt_conversions'] * 2) + (qb_df['receiving_2pt_conversions'] * 2)\
- (qb_df['interceptions']) - (qb_df['sack_fumbles_lost'] * 2) - (qb_df['receiving_fumbles_lost'] * 2) - (qb_df['rushing_fumbles_lost'] * 2)

In [47]:
qb_df['DK_Pts'] = (qb_df['passing_yards'] * 0.04) + (qb_df['rushing_tds'] * 6) + (qb_df['rushing_yards'] * 0.1)\
+ (qb_df['passing_tds'] * 4) + (qb_df['receiving_yards'] * 0.1) + (qb_df['receiving_tds'] * 6) + (qb_df['receptions'] * 1)\
+ (qb_df['rushing_2pt_conversions'] * 2) + (qb_df['passing_2pt_conversions'] * 2) + (qb_df['receiving_2pt_conversions'] * 2)\
- (qb_df['interceptions']) - (qb_df['sack_fumbles_lost']) - (qb_df['receiving_fumbles_lost']) - (qb_df['rushing_fumbles_lost'])


Adding DraftKings bonus points

In [48]:
qb_df['DK_Pts'] = np.where(qb_df['passing_yards'] >= 300, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])
qb_df['DK_Pts'] = np.where(qb_df['receiving_yards'] >= 100, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])
qb_df['DK_Pts'] = np.where(qb_df['rushing_yards'] >= 100, qb_df['DK_Pts'] + 3, qb_df['DK_Pts'])

In [49]:
#pacr = passing air conversion ratio (passing yards/air yards)

In [50]:
qb_df.reset_index(inplace = True)

In [51]:
qb_df.tail()

,season,week,player_display_name,recent_team,opponent_team,position,completions,attempts,passing_yards,passing_tds,...,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts
1701,2024,7,Hendon Hooker,DET,MIN,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1702,2024,7,Trace McSorley,WAS,CAR,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1703,2024,7,Jason Bean,IND,MIA,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1704,2024,7,Ben DiNucci,BUF,TEN,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705,2024,7,JJ McCarthy,MIN,DET,QB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#fanduel_df.head()

In [53]:
#draftkings_df.head()

In [54]:
#We need to group QBs by team because for now we're trying to get DvP means for teams against QBs
grouped_pts = qb_df.groupby(['season', 'week', 'recent_team', 'opponent_team'])[['DK_Pts', 'FD_Pts']].sum().round(3)

In [55]:
#grouped_pts[(grouped_pts['opponent_team'] == 'CLE') & (grouped_pts['season'] == 2001) & (grouped_pts['week'] == 11)]

In [56]:
grouped_pts.reset_index(inplace = True)

In [57]:
grouped_pts

,season,week,recent_team,opponent_team,DK_Pts,FD_Pts
0,2022,1,ARI,KC,23.10,23.10
1,2022,1,ATL,NO,20.80,19.80
2,2022,1,BAL,NYJ,21.22,21.22
3,2022,1,BUF,LAR,33.48,33.48
4,2022,1,CAR,CLE,19.00,19.00
...,...,...,...,...,...,...
1339,2024,7,PIT,NYJ,0.00,0.00
1340,2024,7,SEA,ATL,0.00,0.00
1341,2024,7,SF,KC,0.00,0.00
1342,2024,7,TEN,BUF,0.00,0.00


In [58]:
grouped_pts = grouped_pts.sort_values(by = ['opponent_team', 'season', 'week'])
grouped_pts['opp_game_num'] = grouped_pts.groupby(['opponent_team']).cumcount() + 1
grouped_pts.reset_index(inplace = True)

In [59]:
grouped_pts_raw = grouped_pts.copy()

In [60]:
grouped_pts.drop(columns = ['index'], inplace = True)

# DvP variables
We'll take an 8-game rolling mean for fantasy points allowed to QBs for each team, even if the games go back to last season. DvP means defense vs. position.

In [61]:
def calculate_equal_rolling_mean(group, cols, suffix):
    """
    This function calculates a rolling mean for the last eight games, going back to previous season if necessary.
    It also calculates when there are less than eight games to use.
    """
    for col in cols:
        group[f'{col}{suffix}'] = (
            group[col].shift().rolling(window=8, min_periods=1).mean()
        )
    return group

In [62]:
grouped_pts = grouped_pts.groupby('opponent_team', as_index = False).apply(calculate_equal_rolling_mean, cols=['FD_Pts', 'DK_Pts'], suffix = '_DvP')

In [63]:
grouped_pts.drop(columns = ['DK_Pts', 'FD_Pts', 'opp_game_num'], inplace = True)

In [64]:
qb_df = pd.merge(qb_df, grouped_pts, on = ['season', 'week', 'recent_team', 'opponent_team'], how = 'left')

In [65]:
qb_df.head()

,season,week,player_display_name,recent_team,opponent_team,position,completions,attempts,passing_yards,passing_tds,...,receiving_fumbles,receiving_fumbles_lost,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP
0,2022,1,Tom Brady,TB,DAL,QB,18.0,27.0,212.0,1.0,...,0.0,0.0,0.0,NaN,10.38,10.38,11.38,11.38,NaN,NaN
1,2022,2,Tom Brady,TB,NO,QB,18.0,34.0,190.0,1.0,...,0.0,0.0,0.0,NaN,9.40,9.40,9.40,10.40,19.800000,20.800
2,2022,3,Tom Brady,TB,GB,QB,31.0,42.0,271.0,1.0,...,0.0,0.0,0.0,NaN,14.74,14.74,14.74,14.74,14.440000,14.440
3,2022,4,Tom Brady,TB,KC,QB,39.0,52.0,385.0,3.0,...,0.0,0.0,0.0,NaN,25.40,25.40,25.40,29.40,20.946667,22.280
4,2022,5,Tom Brady,TB,ATL,QB,35.0,52.0,351.0,1.0,...,0.0,0.0,0.0,NaN,19.74,19.74,19.74,22.74,23.300000,24.175


In [66]:
qb_points_last_week = qb_df[(qb_df['season'] == season) & (qb_df['week'] == week - 1)]\
[['season', 'week', 'player_display_name', 'recent_team', 'opponent_team', 'position', 'FD_Pts', 'DK_Pts']]

In [67]:
qb_points_last_week.to_csv('qb_target_' + str(season) + '_' + str(week - 1) + '.csv')

In [68]:
# conn = sqlite3.connect('nfl_dfs.db')

# # Specify the table name
# table_name = 'pbp_non_defense'

# # Query the table and load it into a pandas DataFrame
# pbp_df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

# # Display the DataFrame
# print(pbp_df)

# # Close the connection
# conn.close()

In [69]:
pbp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115012 entries, 0 to 115011
Columns: 372 entries, play_id to pass_oe
dtypes: float64(182), int64(39), object(151)
memory usage: 326.4+ MB


In [70]:
qb_df.reset_index(inplace = True)

In [71]:
qb_df.rename(columns = {'recent_team': 'posteam'}, inplace = True)

# Introducing role variables
We brought in play-by-play data to derive our role variable. For games where more than one quarterback took a snap for a team, we're going to see which one took more snaps and give that QB a QB_role value of 1. All other QBs will have a value of 2. This technique will take on greater importance when we do RBs and WRs. In the case of QBs, if a backup quarterback becomes the starter, we'll change the role value to 1, and that should provide a prediction that's more reflective of the QBs greater role.<br>

In [72]:
# Assuming passer_df is already created
passer_df = pbp_df.groupby(['season', 'week', 'posteam', 'passer_player_id'])['play_id'].count().reset_index()

In [73]:
# Sort by season, week, posteam, and the number of plays in descending order
passer_df = passer_df.sort_values(by=['season', 'week', 'posteam', 'play_id'], ascending=[True, True, True, False])

In [74]:
# Create a rank column to identify the top two QBs
passer_df['QB_role'] = passer_df.groupby(['season', 'week', 'posteam'])['play_id'].rank(method='first', ascending=False)

In [75]:
# Assign 'QB1' or 'QB2' based on the rank
#passer_df['QB_role'] = passer_df['rank'].apply(lambda x: 'QB1' if x == 1 else ('QB2' if x == 2 else None))

In [76]:
# Drop the rank column if you don't need it anymore
#passer_df = passer_df.drop(columns=['rank'])

In [77]:
#passer_df.rename(columns = {'passer_player_id': 'player_id'}, inplace = True)

In [78]:
passer_df.drop(columns = ['play_id'], inplace = True)

In [79]:
passer_df.head()

,season,week,posteam,passer_player_id,QB_role
1,2022,1,ARI,00-0035228,1.0
0,2022,1,ARI,00-0035146,2.0
2,2022,1,ATL,00-0032268,1.0
3,2022,1,BAL,00-0034796,1.0
4,2022,1,BUF,00-0034857,1.0


In [80]:
passer_df.rename(columns = {'passer_player_id': 'player_id', 'posteam': 'recent_team'}, inplace = True)

We need to merge what we get from the play-by-play data back into the weekly data since the weekly data uses full names, and the pbp only uses first initial and last name.

In [81]:
merge_df = weekly_df[weekly_df['position'] == 'QB']

In [82]:
merge_df = merge_df[['player_id', 'season', 'week', 'recent_team', 'player_display_name']]

In [83]:
merge_df = pd.merge(merge_df, passer_df, on = ['season', 'week', 'recent_team', 'player_id'], how = 'left')

In [84]:
merge_df

,player_id,season,week,recent_team,player_display_name,QB_role
0,00-0019596,2022,1,TB,Tom Brady,1.0
1,00-0019596,2022,2,TB,Tom Brady,1.0
2,00-0019596,2022,3,TB,Tom Brady,1.0
3,00-0019596,2022,4,TB,Tom Brady,1.0
4,00-0019596,2022,5,TB,Tom Brady,1.0
...,...,...,...,...,...,...
1604,00-0039918,2024,2,CHI,Caleb Williams,1.0
1605,00-0039918,2024,3,CHI,Caleb Williams,1.0
1606,00-0039918,2024,4,CHI,Caleb Williams,1.0
1607,00-0039918,2024,5,CHI,Caleb Williams,1.0


In [85]:
merge_df.rename(columns = {'recent_team' : 'posteam'}, inplace = True)
#merge_df.drop(columns = ['player_id'], inplace = True)

# Merging
Now we'll merge so that our QB_role variable is added to the dataframe. This is where we also change the name of the dataframe to **quarterback_df.**

In [86]:
quarterback_df = pd.merge(qb_df, merge_df, on = ['season', 'week', 'posteam', 'player_display_name'], how = 'left')

In [87]:
quarterback_df.tail()

,index,season,week,player_display_name,posteam,opponent_team,position,completions,attempts,passing_yards,...,receiving_2pt_conversions,racr,fantasy_points,fantasy_points_ppr,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,player_id,QB_role
1701,1701,2024,7,Hendon Hooker,DET,MIN,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.4475,19.6975,NaN,NaN
1702,1702,2024,7,Trace McSorley,WAS,CAR,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.9975,16.5600,NaN,NaN
1703,1703,2024,7,Jason Bean,IND,MIA,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16.2675,17.1425,NaN,NaN
1704,1704,2024,7,Ben DiNucci,BUF,TEN,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.3300,15.4550,NaN,NaN
1705,1705,2024,7,JJ McCarthy,MIN,DET,QB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.3425,19.8425,NaN,NaN


In [88]:
#quarterback_df = quarterback_df.dropna(subset=['QB_role'])

# Paring down some of the columns
We're drop columns that won't be needed for features.

In [89]:
cols_to_keep = ['season', 'week', 'player_id', 'player_display_name', 'posteam',
       'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds',
       'interceptions', 'sacks', 'sack_fumbles_lost', 'passing_air_yards', 
        'pacr',  'carries', 'rushing_yards', 'rushing_tds', 'FD_Pts', 'DK_Pts', 'FD_Pts_DvP', 'DK_Pts_DvP', 'QB_role']

In [90]:
quarterback_df = quarterback_df[cols_to_keep]

In [91]:
#quarterback_df[quarterback_df['season'] >= 2006]['passing_air_yards']

In [92]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1706 entries, 0 to 1705
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1706 non-null   int64  
 1   week                 1706 non-null   int64  
 2   player_id            1565 non-null   object 
 3   player_display_name  1706 non-null   object 
 4   posteam              1706 non-null   object 
 5   opponent_team        1706 non-null   object 
 6   completions          1609 non-null   float64
 7   attempts             1609 non-null   float64
 8   passing_yards        1609 non-null   float64
 9   passing_tds          1609 non-null   float64
 10  interceptions        1609 non-null   float64
 11  sacks                1609 non-null   float64
 12  sack_fumbles_lost    1609 non-null   float64
 13  passing_air_yards    1609 non-null   float64
 14  pacr                 1535 non-null   float64
 15  carries              1609 non-null   f

# Checking missing on previous weeks
Before we check missing values, we should temporarily break up the dataframe so that the current week, which we're trying to fill, is separate from the rest of the data. Then we can deal with missing values for previous weeks before bringing back the rows we want to fill.

In [93]:
qb_df_temp = quarterback_df[~((quarterback_df['season'] == season) & (quarterback_df['week'] == week))]

In [94]:
this_week = quarterback_df[(quarterback_df['season'] == season) & (quarterback_df['week'] == week)]

In [95]:
qb_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1609 entries, 0 to 1608
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1609 non-null   int64  
 1   week                 1609 non-null   int64  
 2   player_id            1565 non-null   object 
 3   player_display_name  1609 non-null   object 
 4   posteam              1609 non-null   object 
 5   opponent_team        1609 non-null   object 
 6   completions          1609 non-null   float64
 7   attempts             1609 non-null   float64
 8   passing_yards        1609 non-null   float64
 9   passing_tds          1609 non-null   float64
 10  interceptions        1609 non-null   float64
 11  sacks                1609 non-null   float64
 12  sack_fumbles_lost    1609 non-null   float64
 13  passing_air_yards    1609 non-null   float64
 14  pacr                 1535 non-null   float64
 15  carries              1609 non-null   float6

In [96]:
this_week.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 1609 to 1705
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               97 non-null     int64  
 1   week                 97 non-null     int64  
 2   player_id            0 non-null      object 
 3   player_display_name  97 non-null     object 
 4   posteam              97 non-null     object 
 5   opponent_team        97 non-null     object 
 6   completions          0 non-null      float64
 7   attempts             0 non-null      float64
 8   passing_yards        0 non-null      float64
 9   passing_tds          0 non-null      float64
 10  interceptions        0 non-null      float64
 11  sacks                0 non-null      float64
 12  sack_fumbles_lost    0 non-null      float64
 13  passing_air_yards    0 non-null      float64
 14  pacr                 0 non-null      float64
 15  carries              0 non-null      float

In [97]:
# quarterback_df['passing_air_yards'] = np.where((quarterback_df['season'] <= 2005) & (quarterback_df['QB_role'] == 1),\
#                                                quarterback_df['passing_air_yards'].mean(), quarterback_df['passing_air_yards'])

# Filling pacr
pacr is passing yards/air yards, and every QB will have at least some air yards even if he doesn't have passing yards. So we were going to just fill missing pacr values with 0, but upon further review, we thought about filling with 1. Then we saw the mean is almost 1 (0.93) and the median is .86. So we'll just fill with the mean

In [98]:
qb_df_temp['pacr'] = qb_df_temp['pacr'].fillna(qb_df_temp['pacr'].mean())

C:\Users\Owner\AppData\Local\Temp\ipykernel_35456\52539329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_df_temp['pacr'] = qb_df_temp['pacr'].fillna(qb_df_temp['pacr'].mean())


Let's see if dropping all 2022 Week 1 rows will take care of some of the missing values. We obviously don't need that for L8 variables.

In [99]:
# quarterback_df['FD_Pts_DvP'] = quarterback_df['FD_Pts_DvP'].fillna(quarterback_df['FD_Pts_DvP'].mean())
# quarterback_df['DK_Pts_DvP'] = quarterback_df['DK_Pts_DvP'].fillna(quarterback_df['DK_Pts_DvP'].mean())
qb_df_temp = qb_df_temp[~((qb_df_temp['season'] == 2022) & (qb_df_temp['week'] == 1))]

In [100]:
qb_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1572 entries, 1 to 1608
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1572 non-null   int64  
 1   week                 1572 non-null   int64  
 2   player_id            1528 non-null   object 
 3   player_display_name  1572 non-null   object 
 4   posteam              1572 non-null   object 
 5   opponent_team        1572 non-null   object 
 6   completions          1572 non-null   float64
 7   attempts             1572 non-null   float64
 8   passing_yards        1572 non-null   float64
 9   passing_tds          1572 non-null   float64
 10  interceptions        1572 non-null   float64
 11  sacks                1572 non-null   float64
 12  sack_fumbles_lost    1572 non-null   float64
 13  passing_air_yards    1572 non-null   float64
 14  pacr                 1572 non-null   float64
 15  carries              1572 non-null   float6

In [101]:
this_week.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 1609 to 1705
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               97 non-null     int64  
 1   week                 97 non-null     int64  
 2   player_id            0 non-null      object 
 3   player_display_name  97 non-null     object 
 4   posteam              97 non-null     object 
 5   opponent_team        97 non-null     object 
 6   completions          0 non-null      float64
 7   attempts             0 non-null      float64
 8   passing_yards        0 non-null      float64
 9   passing_tds          0 non-null      float64
 10  interceptions        0 non-null      float64
 11  sacks                0 non-null      float64
 12  sack_fumbles_lost    0 non-null      float64
 13  passing_air_yards    0 non-null      float64
 14  pacr                 0 non-null      float64
 15  carries              0 non-null      float

In [102]:
quarterback_df = pd.concat([qb_df_temp, this_week], axis = 0)

In [103]:
#With the dataframes back together, we can universally fill in missing QB_role values with 2
quarterback_df['QB_role'] = quarterback_df['QB_role'].fillna(2)

In [104]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1669 entries, 1 to 1705
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   season               1669 non-null   int64  
 1   week                 1669 non-null   int64  
 2   player_id            1528 non-null   object 
 3   player_display_name  1669 non-null   object 
 4   posteam              1669 non-null   object 
 5   opponent_team        1669 non-null   object 
 6   completions          1572 non-null   float64
 7   attempts             1572 non-null   float64
 8   passing_yards        1572 non-null   float64
 9   passing_tds          1572 non-null   float64
 10  interceptions        1572 non-null   float64
 11  sacks                1572 non-null   float64
 12  sack_fumbles_lost    1572 non-null   float64
 13  passing_air_yards    1572 non-null   float64
 14  pacr                 1572 non-null   float64
 15  carries              1572 non-null   float6

In [105]:
quarterback_df.drop(columns = ['player_id'], inplace = True)

In [106]:
missing_df = quarterback_df[quarterback_df.isna().any(axis = 1)]

In [107]:
missing_df['season'].value_counts()

season
2024    97
Name: count, dtype: int64

In [108]:
missing_df['week'].value_counts()

week
7    97
Name: count, dtype: int64

In [109]:
cols_L8 = ['completions', 'attempts', 'passing_yards', 'passing_tds','interceptions',\
           'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds']

In [110]:
qb_L8_features = quarterback_df.groupby(['player_display_name', 'season', 'week'])[cols_L8].sum()

In [111]:
qb_L8_features

completions  attempts  passing_yards  \
player_display_name season week                                         
AJ McCarron         2023   14            1.0       1.0           -1.0   
                           18            3.0       4.0           20.0   
Aaron Rodgers       2022   2            19.0      25.0          234.0   
                           3            27.0      35.0          255.0   
                           4            21.0      35.0          251.0   
...                                      ...       ...            ...   
Zach Wilson         2023   9            33.0      49.0          263.0   
                           10           23.0      39.0          263.0   
                           11            7.0      15.0           81.0   
                           14           27.0      36.0          301.0   
                           15            4.0      11.0           26.0   

                                 passing_tds  interceptions  sacks  \
player_display_name season week                                      
AJ McCarron         2023   14            0.0            0.0    0.0   
                           18            0.0            0.0    1.0   
Aaron Rodgers       2022   2             2.0            0.0    3.0   
                           3             2.0            1.0    1.0   
                           4             2.0            1.0    1.0   
...                                      ...            ...    ...   
Zach Wilson         2023   9             0.0            0.0    8.0   
                           10            0.0            1.0    2.0   
                           11            1.0            1.0    5.0   
                           14            2.0            0.0    4.0   
                           15            0.0            0.0    4.0   

                                 passing_air_yards      pacr  carries  \
player_display_name season week                                         
AJ McCarron         2023   14                 -1.0  0.000000      0.0   
                           18                 -7.0  0.000000      0.0   
Aaron Rodgers       2022   2                 139.0  1.683453      5.0   
                           3                 177.0  1.440678      1.0   
                           4                 357.0  0.703081      1.0   
...                                            ...       ...      ...   
Zach Wilson         2023   9                 353.0  0.745042      2.0   
                           10                358.0  0.734637      4.0   
                           11                 76.0  1.065789      4.0   
                           14                238.0  1.264706      3.0   
                           15                 29.0  0.896552      0.0   

                                 rushing_yards  rushing_tds  
player_display_name season week                              
AJ McCarron         2023   14              0.0          0.0  
                           18              0.0          0.0  
Aaron Rodgers       2022   2              10.0          0.0  
                           3              -1.0          0.0  
                           4               1.0          0.0  
...                                        ...          ...  
Zach Wilson         2023   9               7.0          0.0  
                           10             54.0          0.0  
                           11             15.0          0.0  
                           14             12.0          0.0  
                           15              0.0          0.0  

[1669 rows x 11 columns]

# L8 variables
L8 variables are rolling means of features over the last eight games that each QB has played. Just like we did for the DvP variables, we'll calculate features over the previous eight games for individual QBs.

In [112]:
qb_L8_features = qb_L8_features.sort_values(by = ['player_display_name', 'season', 'week'])
qb_L8_features['game_num'] = qb_L8_features.groupby(['player_display_name', 'season']).cumcount() + 1
#quarterback_df.reset_index(drop = True, inplace = True)

In [113]:
qb_L8_features

completions  attempts  passing_yards  \
player_display_name season week                                         
AJ McCarron         2023   14            1.0       1.0           -1.0   
                           18            3.0       4.0           20.0   
Aaron Rodgers       2022   2            19.0      25.0          234.0   
                           3            27.0      35.0          255.0   
                           4            21.0      35.0          251.0   
...                                      ...       ...            ...   
Zach Wilson         2023   9            33.0      49.0          263.0   
                           10           23.0      39.0          263.0   
                           11            7.0      15.0           81.0   
                           14           27.0      36.0          301.0   
                           15            4.0      11.0           26.0   

                                 passing_tds  interceptions  sacks  \
player_display_name season week                                      
AJ McCarron         2023   14            0.0            0.0    0.0   
                           18            0.0            0.0    1.0   
Aaron Rodgers       2022   2             2.0            0.0    3.0   
                           3             2.0            1.0    1.0   
                           4             2.0            1.0    1.0   
...                                      ...            ...    ...   
Zach Wilson         2023   9             0.0            0.0    8.0   
                           10            0.0            1.0    2.0   
                           11            1.0            1.0    5.0   
                           14            2.0            0.0    4.0   
                           15            0.0            0.0    4.0   

                                 passing_air_yards      pacr  carries  \
player_display_name season week                                         
AJ McCarron         2023   14                 -1.0  0.000000      0.0   
                           18                 -7.0  0.000000      0.0   
Aaron Rodgers       2022   2                 139.0  1.683453      5.0   
                           3                 177.0  1.440678      1.0   
                           4                 357.0  0.703081      1.0   
...                                            ...       ...      ...   
Zach Wilson         2023   9                 353.0  0.745042      2.0   
                           10                358.0  0.734637      4.0   
                           11                 76.0  1.065789      4.0   
                           14                238.0  1.264706      3.0   
                           15                 29.0  0.896552      0.0   

                                 rushing_yards  rushing_tds  game_num  
player_display_name season week                                        
AJ McCarron         2023   14              0.0          0.0         1  
                           18              0.0          0.0         2  
Aaron Rodgers       2022   2              10.0          0.0         1  
                           3              -1.0          0.0         2  
                           4               1.0          0.0         3  
...                                        ...          ...       ...  
Zach Wilson         2023   9               7.0          0.0         8  
                           10             54.0          0.0         9  
                           11             15.0          0.0        10  
                           14             12.0          0.0        11  
                           15              0.0          0.0        12  

[1669 rows x 12 columns]

In [114]:
qb_L8_features = qb_L8_features.groupby(['player_display_name'], as_index = False).apply(calculate_equal_rolling_mean, cols=cols_L8, suffix = '_L8')

In [115]:
qb_L8_features.reset_index(inplace = True)

In [116]:
qb_L8_features

,level_0,player_display_name,season,week,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8
0,0,AJ McCarron,2023,14,1.0,1.0,-1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,AJ McCarron,2023,18,3.0,4.0,20.0,0.0,0.0,1.0,...,1.000,-1.000,0.000,0.000,0.000,-1.000,0.000000,0.000,0.000,0.0
2,1,Aaron Rodgers,2022,2,19.0,25.0,234.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aaron Rodgers,2022,3,27.0,35.0,255.0,2.0,1.0,1.0,...,25.000,234.000,2.000,0.000,3.000,139.000,1.683453,5.000,10.000,0.0
4,1,Aaron Rodgers,2022,4,21.0,35.0,251.0,2.0,1.0,1.0,...,30.000,244.500,2.000,0.500,2.000,158.000,1.562066,3.000,4.500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664,140,Zach Wilson,2023,9,33.0,49.0,263.0,0.0,0.0,8.0,...,29.500,178.625,0.625,0.750,3.250,224.875,0.845958,3.000,15.500,0.0
1665,140,Zach Wilson,2023,10,23.0,39.0,263.0,0.0,1.0,2.0,...,33.375,200.000,0.625,0.625,3.875,246.375,0.875553,3.125,16.250,0.0
1666,140,Zach Wilson,2023,11,7.0,15.0,81.0,1.0,1.0,5.0,...,35.625,215.375,0.500,0.625,3.875,279.125,0.785091,3.125,22.250,0.0
1667,140,Zach Wilson,2023,14,27.0,36.0,301.0,2.0,0.0,4.0,...,34.125,204.250,0.500,0.375,4.125,254.500,0.840476,3.000,19.625,0.0


In [117]:
qb_L8_features.drop(columns = ['level_0', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',\
                              'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds'], inplace = True)

In [118]:
quarterback_df = pd.merge(quarterback_df, qb_L8_features, on = ['player_display_name', 'season', 'week'], how = 'left')

In [119]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1669 non-null   int64  
 1   week                  1669 non-null   int64  
 2   player_display_name   1669 non-null   object 
 3   posteam               1669 non-null   object 
 4   opponent_team         1669 non-null   object 
 5   completions           1572 non-null   float64
 6   attempts              1572 non-null   float64
 7   passing_yards         1572 non-null   float64
 8   passing_tds           1572 non-null   float64
 9   interceptions         1572 non-null   float64
 10  sacks                 1572 non-null   float64
 11  sack_fumbles_lost     1572 non-null   float64
 12  passing_air_yards     1572 non-null   float64
 13  pacr                  1572 non-null   float64
 14  carries               1572 non-null   float64
 15  rushing_yards        

In [120]:
quarterback_df[(quarterback_df.isna().any(axis = 1))]

,season,week,player_display_name,posteam,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,...,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8
0,2022,2,Tom Brady,TB,NO,18.0,34.0,190.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2022,2,Aaron Rodgers,GB,CHI,19.0,25.0,234.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,2022,2,Matt Ryan,IND,JAX,16.0,30.0,195.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2022,2,Joe Flacco,NYJ,CLE,26.0,44.0,307.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2022,7,Chad Henne,KC,SF,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664,2024,7,Hendon Hooker,DET,MIN,NaN,NaN,NaN,NaN,NaN,...,2.00,19.0,0.0,0.00,1.0,10.00,1.900000,1.0,-1.00,0.0
1665,2024,7,Trace McSorley,WAS,CAR,NaN,NaN,NaN,NaN,NaN,...,19.75,100.0,0.0,1.25,0.5,156.75,0.631779,3.0,10.25,0.0
1666,2024,7,Jason Bean,IND,MIA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1667,2024,7,Ben DiNucci,BUF,TEN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
quarterback_df.drop(columns = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',\
                              'sacks', 'passing_air_yards', 'pacr', 'carries', 'rushing_yards', 'rushing_tds',\
                              'sack_fumbles_lost', 'game_num'], inplace = True)

In [122]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1669 non-null   int64  
 1   week                  1669 non-null   int64  
 2   player_display_name   1669 non-null   object 
 3   posteam               1669 non-null   object 
 4   opponent_team         1669 non-null   object 
 5   FD_Pts                1572 non-null   float64
 6   DK_Pts                1572 non-null   float64
 7   FD_Pts_DvP            1669 non-null   float64
 8   DK_Pts_DvP            1669 non-null   float64
 9   QB_role               1669 non-null   float64
 10  completions_L8        1528 non-null   float64
 11  attempts_L8           1528 non-null   float64
 12  passing_yards_L8      1528 non-null   float64
 13  passing_tds_L8        1528 non-null   float64
 14  interceptions_L8      1528 non-null   float64
 15  sacks_L8             

In [123]:
cols_to_check = ['completions_L8', 'attempts_L8', 'passing_yards_L8', 'passing_tds_L8',\
                 'interceptions_L8', 'sacks_L8', 'passing_air_yards_L8', 'pacr_L8',\
                 'carries_L8', 'rushing_yards_L8', 'rushing_tds_L8']

In [124]:
missing_L8 = quarterback_df[quarterback_df[cols_to_check].isna().any(axis = 1)]

In [125]:
#pd.set_option('display.max_rows', None)

In [126]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1669 non-null   int64  
 1   week                  1669 non-null   int64  
 2   player_display_name   1669 non-null   object 
 3   posteam               1669 non-null   object 
 4   opponent_team         1669 non-null   object 
 5   FD_Pts                1572 non-null   float64
 6   DK_Pts                1572 non-null   float64
 7   FD_Pts_DvP            1669 non-null   float64
 8   DK_Pts_DvP            1669 non-null   float64
 9   QB_role               1669 non-null   float64
 10  completions_L8        1528 non-null   float64
 11  attempts_L8           1528 non-null   float64
 12  passing_yards_L8      1528 non-null   float64
 13  passing_tds_L8        1528 non-null   float64
 14  interceptions_L8      1528 non-null   float64
 15  sacks_L8             

# Must fill missing values
We need to run the new data through the model, so we can't have missing data. We'll fill with the mean.

In [127]:
columns_to_fill = [
    'completions_L8', 'attempts_L8', 'passing_yards_L8', 'passing_tds_L8',
    'interceptions_L8', 'sacks_L8', 'passing_air_yards_L8', 'pacr_L8',
    'carries_L8', 'rushing_yards_L8', 'rushing_tds_L8'
]

In [128]:
quarterback_df[columns_to_fill] = quarterback_df[columns_to_fill].apply(lambda col: col.fillna(col.mean()))

In [129]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                1669 non-null   int64  
 1   week                  1669 non-null   int64  
 2   player_display_name   1669 non-null   object 
 3   posteam               1669 non-null   object 
 4   opponent_team         1669 non-null   object 
 5   FD_Pts                1572 non-null   float64
 6   DK_Pts                1572 non-null   float64
 7   FD_Pts_DvP            1669 non-null   float64
 8   DK_Pts_DvP            1669 non-null   float64
 9   QB_role               1669 non-null   float64
 10  completions_L8        1669 non-null   float64
 11  attempts_L8           1669 non-null   float64
 12  passing_yards_L8      1669 non-null   float64
 13  passing_tds_L8        1669 non-null   float64
 14  interceptions_L8      1669 non-null   float64
 15  sacks_L8             

# Percentage variables
We'll take our L8 means and derive certain percentages.

In [130]:
quarterback_df['comp_pct'] = quarterback_df['completions_L8']/quarterback_df['attempts_L8']
quarterback_df['yds_per_attempt'] = quarterback_df['passing_yards_L8']/quarterback_df['attempts_L8']
quarterback_df['td_pct'] = quarterback_df['passing_tds_L8']/quarterback_df['attempts_L8']
quarterback_df['sack_pct'] = quarterback_df['sacks_L8']/quarterback_df['attempts_L8']
quarterback_df['int_pct'] = quarterback_df['interceptions_L8']/quarterback_df['attempts_L8']

In [131]:
quarterback_df.tail(10)

,season,week,player_display_name,posteam,opponent_team,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,QB_role,...,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8,comp_pct,yds_per_attempt,td_pct,sack_pct,int_pct
1659,2024,7,Adrian Martinez,NYJ,PIT,NaN,NaN,15.8225,16.3225,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1660,2024,7,Kyle Allen,PIT,NYJ,NaN,NaN,11.5650,12.1900,2.0,...,45.375000,0.982809,1.875000,-1.500000,0.000000,0.525000,5.500000,0.025000,0.000000,0.050000
1661,2024,7,Carter Bradley,LV,LAR,NaN,NaN,18.0675,18.8175,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1662,2024,7,Kenny Pickett,PHI,NYG,NaN,NaN,16.5525,17.3025,2.0,...,176.875000,0.958377,4.000000,4.500000,0.125000,0.629442,6.431472,0.010152,0.060914,0.000000
1663,2024,7,Anthony Brown,BUF,TEN,NaN,NaN,15.3300,15.4550,2.0,...,200.500000,0.563588,1.500000,-2.500000,0.000000,0.448980,6.163265,0.000000,0.102041,0.040816
1664,2024,7,Hendon Hooker,DET,MIN,NaN,NaN,18.4475,19.6975,2.0,...,10.000000,1.900000,1.000000,-1.000000,0.000000,0.500000,9.500000,0.000000,0.500000,0.000000
1665,2024,7,Trace McSorley,WAS,CAR,NaN,NaN,15.9975,16.5600,2.0,...,156.750000,0.631779,3.000000,10.250000,0.000000,0.544304,5.063291,0.000000,0.025316,0.063291
1666,2024,7,Jason Bean,IND,MIA,NaN,NaN,16.2675,17.1425,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1667,2024,7,Ben DiNucci,BUF,TEN,NaN,NaN,15.3300,15.4550,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1668,2024,7,JJ McCarthy,MIN,DET,NaN,NaN,18.3425,19.8425,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453


In [132]:
quarterback_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [133]:
mean_dict = quarterback_df.mean(numeric_only=True).to_dict()

In [134]:
mean_dict

{'season': 2022.8100659077293,
 'week': 9.179149191132415,
 'FD_Pts': 14.039211195928752,
 'DK_Pts': 14.712557251908397,
 'FD_Pts_DvP': 17.142511783503092,
 'DK_Pts_DvP': 17.963417872121887,
 'QB_role': 1.2480527261833434,
 'completions_L8': 17.891299707055595,
 'attempts_L8': 27.74879705809025,
 'passing_yards_L8': 195.20496447270006,
 'passing_tds_L8': 1.1430916542009473,
 'interceptions_L8': 0.6230335327848416,
 'sacks_L8': 2.024214659685864,
 'passing_air_yards_L8': 215.50188310271756,
 'pacr_L8': 0.9596623164767317,
 'carries_L8': 3.6659545624532535,
 'rushing_yards_L8': 16.095326134380453,
 'rushing_tds_L8': 0.1817618735976066,
 'comp_pct': 0.6384350629053981,
 'yds_per_attempt': 7.002896111938334,
 'td_pct': 0.04114584509684851,
 'sack_pct': 0.07827686758552337,
 'int_pct': 0.023867748630909763}

In [135]:
columns_to_fill = ['comp_pct', 'yds_per_attempt', 'td_pct', 'sack_pct', 'int_pct']

In [136]:
quarterback_df[columns_to_fill] = quarterback_df[columns_to_fill].apply(lambda col: col.fillna(col.mean()))

In [137]:
# quarterback_df.drop(columns = ['completions', 'attempts', 'passing_yards', 'passing_tds', 'sacks', 'interceptions',\
#                               'sack_fumbles_lost', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'game_num'], inplace = True)

In [138]:
quarterback_df.tail(10)

,season,week,player_display_name,posteam,opponent_team,FD_Pts,DK_Pts,FD_Pts_DvP,DK_Pts_DvP,QB_role,...,passing_air_yards_L8,pacr_L8,carries_L8,rushing_yards_L8,rushing_tds_L8,comp_pct,yds_per_attempt,td_pct,sack_pct,int_pct
1659,2024,7,Adrian Martinez,NYJ,PIT,NaN,NaN,15.8225,16.3225,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1660,2024,7,Kyle Allen,PIT,NYJ,NaN,NaN,11.5650,12.1900,2.0,...,45.375000,0.982809,1.875000,-1.500000,0.000000,0.525000,5.500000,0.025000,0.000000,0.050000
1661,2024,7,Carter Bradley,LV,LAR,NaN,NaN,18.0675,18.8175,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1662,2024,7,Kenny Pickett,PHI,NYG,NaN,NaN,16.5525,17.3025,2.0,...,176.875000,0.958377,4.000000,4.500000,0.125000,0.629442,6.431472,0.010152,0.060914,0.000000
1663,2024,7,Anthony Brown,BUF,TEN,NaN,NaN,15.3300,15.4550,2.0,...,200.500000,0.563588,1.500000,-2.500000,0.000000,0.448980,6.163265,0.000000,0.102041,0.040816
1664,2024,7,Hendon Hooker,DET,MIN,NaN,NaN,18.4475,19.6975,2.0,...,10.000000,1.900000,1.000000,-1.000000,0.000000,0.500000,9.500000,0.000000,0.500000,0.000000
1665,2024,7,Trace McSorley,WAS,CAR,NaN,NaN,15.9975,16.5600,2.0,...,156.750000,0.631779,3.000000,10.250000,0.000000,0.544304,5.063291,0.000000,0.025316,0.063291
1666,2024,7,Jason Bean,IND,MIA,NaN,NaN,16.2675,17.1425,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1667,2024,7,Ben DiNucci,BUF,TEN,NaN,NaN,15.3300,15.4550,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453
1668,2024,7,JJ McCarthy,MIN,DET,NaN,NaN,18.3425,19.8425,2.0,...,215.501883,0.959662,3.665955,16.095326,0.181762,0.644759,7.034718,0.041194,0.072948,0.022453


# Wind speed, division games, odds
We'll bring in the odds CSVs. This is also where we can break off the data for the current week. We no longer need previous data to derive features.

In [139]:
fd_spreads = pd.read_csv('fd_spreads_' + str(season) + '_' + str(week) + '.csv')
dk_spreads = pd.read_csv('dk_spreads_' + str(season) + '_' + str(week) + '.csv')

In [140]:
fd_spreads.drop(columns = ['Unnamed: 0'], inplace = True)
dk_spreads.drop(columns = ['Unnamed: 0'], inplace = True)

In [141]:
fd_spreads.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,JAX,NE,42.5,1,1,0,-5.5,18.50,24.00,14,2024,7
1,ATL,SEA,51.5,0,0,0,-3.0,24.25,27.25,0,2024,7
2,BUF,TEN,41.5,1,0,0,-8.5,16.50,25.00,8,2024,7
3,CLE,CIN,41.5,1,1,0,5.5,23.50,18.00,6,2024,7
4,MIN,DET,50.5,0,1,0,-1.5,24.50,26.00,0,2024,7


In [142]:
dk_spreads.head()

,opponent,team,total_line,outdoors,grass,home_team,spread_line,pred_total,opp_total,wind,season,week
0,JAX,NE,42.5,1,1,0,-5.5,18.50,24.00,14,2024,7
1,ATL,SEA,51.0,0,0,0,-3.0,24.00,27.00,0,2024,7
2,BUF,TEN,41.0,1,0,0,-9.0,16.00,25.00,8,2024,7
3,CLE,CIN,41.5,1,1,0,6.0,23.75,17.75,6,2024,7
4,MIN,DET,51.0,0,1,0,-1.5,24.75,26.25,0,2024,7


In [143]:
quarterback_df.rename(columns = {'posteam': 'team', 'opponent_team': 'opponent'}, inplace = True)

In [144]:
quarterback_df = quarterback_df[(quarterback_df['season'] == season) & (quarterback_df['week'] == week)]

In [145]:
quarterback_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 1572 to 1668
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                97 non-null     int64  
 1   week                  97 non-null     int64  
 2   player_display_name   97 non-null     object 
 3   team                  97 non-null     object 
 4   opponent              97 non-null     object 
 5   FD_Pts                0 non-null      float64
 6   DK_Pts                0 non-null      float64
 7   FD_Pts_DvP            97 non-null     float64
 8   DK_Pts_DvP            97 non-null     float64
 9   QB_role               97 non-null     float64
 10  completions_L8        97 non-null     float64
 11  attempts_L8           97 non-null     float64
 12  passing_yards_L8      97 non-null     float64
 13  passing_tds_L8        97 non-null     float64
 14  interceptions_L8      97 non-null     float64
 15  sacks_L8              97 

In [146]:
qb_df_fd = pd.merge(quarterback_df, fd_spreads, on = ['team', 'opponent', 'season', 'week'], how = 'left')
qb_df_dk = pd.merge(quarterback_df, dk_spreads, on = ['team', 'opponent', 'season', 'week'], how = 'left')

In [147]:
qb_df_fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                97 non-null     int64  
 1   week                  97 non-null     int64  
 2   player_display_name   97 non-null     object 
 3   team                  97 non-null     object 
 4   opponent              97 non-null     object 
 5   FD_Pts                0 non-null      float64
 6   DK_Pts                0 non-null      float64
 7   FD_Pts_DvP            97 non-null     float64
 8   DK_Pts_DvP            97 non-null     float64
 9   QB_role               97 non-null     float64
 10  completions_L8        97 non-null     float64
 11  attempts_L8           97 non-null     float64
 12  passing_yards_L8      97 non-null     float64
 13  passing_tds_L8        97 non-null     float64
 14  interceptions_L8      97 non-null     float64
 15  sacks_L8              97 

In [148]:
divisions = {
    'CLE': 'AFC North',
    'LAR': 'NFC West',
    'LV': 'AFC West',
    'KC': 'AFC West',
    'CAR': 'NFC South',
    'NYG': 'NFC East',
    'HOU': 'AFC South',
    'DEN': 'AFC West',
    'MIN': 'NFC North',
    'TEN': 'AFC South',
    'JAX': 'AFC South',
    'SEA': 'NFC West',
    'DET': 'NFC North',
    'NO': 'NFC South',
    'CIN': 'AFC North',
    'ATL': 'NFC South',
    'NYJ': 'AFC East',
    'PHI': 'NFC East',
    'DAL': 'NFC East',
    'WAS': 'NFC East',
    'PIT': 'AFC North',
    'ARI': 'NFC West',
    'CHI': 'NFC North',
    'MIA': 'AFC East',
    'BUF': 'AFC East',
    'BAL': 'AFC North',
    'TB': 'NFC South',
    'SF': 'NFC West',
    'LAC': 'AFC West',
    'IND': 'AFC South',
    'GB': 'NFC North',
    'NE': 'AFC East'
}

# You now have a dictionary `divisions` where each team is mapped to its division.


# Adding div_game binary column

In [149]:
# Map the team and opponent columns to their respective divisions
qb_df_fd['team_division'] = qb_df_fd['team'].map(divisions)
qb_df_fd['opponent_division'] = qb_df_fd['opponent'].map(divisions)

# Create the div_game column (1 if they are in the same division, 0 otherwise)
qb_df_fd['div_game'] = np.where(qb_df_fd['team_division'] == qb_df_fd['opponent_division'], 1, 0)

# Optionally, drop the temporary division columns if you don't need them
qb_df_fd.drop(['team_division', 'opponent_division'], axis=1, inplace=True)


In [150]:
# Map the team and opponent columns to their respective divisions
qb_df_dk['team_division'] = qb_df_dk['team'].map(divisions)
qb_df_dk['opponent_division'] = qb_df_dk['opponent'].map(divisions)

# Create the div_game column (1 if they are in the same division, 0 otherwise)
qb_df_dk['div_game'] = np.where(qb_df_dk['team_division'] == qb_df_dk['opponent_division'], 1, 0)

# Optionally, drop the temporary division columns if you don't need them
qb_df_dk.drop(['team_division', 'opponent_division'], axis=1, inplace=True)

In [151]:
qb_df_fd.rename(columns = {'player_display_name': 'name'}, inplace = True)
qb_df_dk.rename(columns = {'player_display_name': 'name'}, inplace = True)

In [152]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
2,108151-62239,Josh Allen,QB,9200,BUF,TEN,1,Active,10-20-2024,7,Josh Allen
3,108151-102785,Jayden Daniels,QB,9100,WAS,CAR,1,Active,10-20-2024,7,Jayden Daniels
8,108151-69017,Jordan Love,QB,8700,GB,HOU,1,Active,10-20-2024,7,Jordan Love
11,108151-69531,Jalen Hurts,QB,8500,PHI,NYG,0,Active,10-20-2024,7,Jalen Hurts
13,108151-129471,CJ Stroud,QB,8400,HOU,GB,0,Active,10-20-2024,7,CJ Stroud


In [153]:
draftkings_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
9,36290724,Jayden Daniels,QB,7600,WAS,CAR,1,Active,10-20-2024,7,Jayden Daniels
11,36290725,Josh Allen,QB,7500,BUF,TEN,1,Active,10-20-2024,7,Josh Allen
17,36290726,Jalen Hurts,QB,7300,PHI,NYG,0,Active,10-20-2024,7,Jalen Hurts
21,36290727,CJ Stroud,QB,7100,HOU,GB,0,Active,10-20-2024,7,CJ Stroud
23,36290728,Patrick Mahomes,QB,7000,KC,SF,0,Active,10-20-2024,7,Patrick Mahomes


In [154]:
fanduel_df.rename(columns = {'player_display_name': 'name'}, inplace = True)
draftkings_df.rename(columns = {'player_display_name': 'name'}, inplace = True)

In [155]:
starting_qbs = [('BUF', 'Josh Allen'), ('MIA', 'Tyler Huntley'), ('NYJ', 'Aaron Rodgers'), ('NE', 'Drake Maye'),\
               ('BAL', 'Lamar Jackson'), ('CIN', 'Joe Burrow'), ('CLE', 'Deshaun Watson'), ('PIT', 'Justin Fields'),\
               ('IND', 'Anthony Richardson'), ('JAX', 'Trevor Lawrence'), ('HOU', 'CJ Stroud'), ('TEN', 'Will Levis'),\
               ('KC', 'Patrick Mahomes'), ('LV', "Aidan O'Connell"), ('LAC', 'Justin Herbert'), ('DEN', 'Bo Nix'),\
               ('DAL', 'Dak Prescott'), ('PHI', 'Jalen Hurts'), ('NYG', 'Daniel Jones'), ('WAS', 'Jayden Daniels'),\
               ('MIN', 'Sam Darnold'), ('DET', 'Jared Goff'), ('GB', 'Jordan Love'), ('CHI', 'Caleb Williams'),\
               ('TB', 'Baker Mayfield'), ('NO', 'Spencer Rattler'), ('CAR', 'Andy Dalton'), ('ATL', 'Kirk Cousins'),\
               ('SF', 'Brock Purdy'), ('SEA', 'Geno Smith'), ('LAR', 'Matthew Stafford'), ('ARI', 'Kyler Murray')]

In [156]:
starting_qbs = pd.DataFrame(data = starting_qbs, columns = ['team', 'name'])

In [157]:
starting_qbs.to_csv('starting_QBs.csv', index = False)

In [158]:
qb_1 = pd.read_csv('starting_QBs.csv')

In [159]:
qb_1

,team,name
0,BUF,Josh Allen
1,MIA,Tyler Huntley
2,NYJ,Aaron Rodgers
3,NE,Drake Maye
4,BAL,Lamar Jackson
5,CIN,Joe Burrow
6,CLE,Deshaun Watson
7,PIT,Justin Fields
8,IND,Anthony Richardson
9,JAX,Trevor Lawrence


In [160]:
qb_df_fd.loc[
    qb_df_fd.set_index(['team', 'name']).index.isin(qb_1.set_index(['team', 'name']).index), 
    'QB_role'
] = 1

In [161]:
qb_df_dk.loc[
    qb_df_dk.set_index(['team', 'name']).index.isin(qb_1.set_index(['team', 'name']).index), 
    'QB_role'
] = 1

In [162]:
qb_df_fd['QB_role'].value_counts()

QB_role
2.0    75
1.0    22
Name: count, dtype: int64

In [163]:
qb_df_dk['QB_role'].value_counts()

QB_role
2.0    75
1.0    22
Name: count, dtype: int64

In [164]:
features = ['FD_Pts_DvP', 'DK_Pts_DvP', 'QB_role', 'completions_L8', 'attempts_L8',
       'passing_yards_L8', 'passing_tds_L8', 'interceptions_L8', 'sacks_L8',
       'passing_air_yards_L8', 'pacr_L8', 'carries_L8', 'rushing_yards_L8',
       'rushing_tds_L8', 'comp_pct', 'yds_per_attempt', 'td_pct', 'sack_pct',
       'int_pct', 'wind', 'div_game', 'spread_line', 'total_line', 'outdoors',
       'grass', 'home_team', 'pred_total', 'opp_total']

In [165]:
qb_df_fd.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)
qb_df_dk.drop(columns = ['FD_Pts', 'DK_Pts'], inplace = True)

In [166]:
qb_df_fd.replace([np.inf, -np.inf], np.nan, inplace=True)
qb_df_dk.replace([np.inf, -np.inf], np.nan, inplace=True)

In [167]:
fanduel_df.head()

,ID,name,position,salary,team,opponent,home_team,status,date,week,matched_name
2,108151-62239,Josh Allen,QB,9200,BUF,TEN,1,Active,10-20-2024,7,Josh Allen
3,108151-102785,Jayden Daniels,QB,9100,WAS,CAR,1,Active,10-20-2024,7,Jayden Daniels
8,108151-69017,Jordan Love,QB,8700,GB,HOU,1,Active,10-20-2024,7,Jordan Love
11,108151-69531,Jalen Hurts,QB,8500,PHI,NYG,0,Active,10-20-2024,7,Jalen Hurts
13,108151-129471,CJ Stroud,QB,8400,HOU,GB,0,Active,10-20-2024,7,CJ Stroud


In [168]:
fanduel_df = fanduel_df[['name', 'position', 'salary', 'team', 'opponent', 'status', 'week']]
draftkings_df = draftkings_df[['name', 'position', 'salary', 'team', 'opponent', 'status', 'week']]

In [169]:
fanduel_df.head()

,name,position,salary,team,opponent,status,week
2,Josh Allen,QB,9200,BUF,TEN,Active,7
3,Jayden Daniels,QB,9100,WAS,CAR,Active,7
8,Jordan Love,QB,8700,GB,HOU,Active,7
11,Jalen Hurts,QB,8500,PHI,NYG,Active,7
13,CJ Stroud,QB,8400,HOU,GB,Active,7


In [170]:
draftkings_df.head()

,name,position,salary,team,opponent,status,week
9,Jayden Daniels,QB,7600,WAS,CAR,Active,7
11,Josh Allen,QB,7500,BUF,TEN,Active,7
17,Jalen Hurts,QB,7300,PHI,NYG,Active,7
21,CJ Stroud,QB,7100,HOU,GB,Active,7
23,Patrick Mahomes,QB,7000,KC,SF,Active,7


# Merging player list with the model features
We need to bring back the salary and status columns. This also will filter out players who aren't on the list for one site or the other.

In [171]:
qb_df_fd = pd.merge(fanduel_df, qb_df_fd, on = ['name', 'team', 'opponent', 'week'], how = 'left')

In [172]:
qb_df_dk = pd.merge(draftkings_df, qb_df_dk, on = ['name', 'team', 'opponent', 'week'], how = 'left')

In [173]:
qb_df_fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  97 non-null     object 
 1   position              97 non-null     object 
 2   salary                97 non-null     int64  
 3   team                  97 non-null     object 
 4   opponent              97 non-null     object 
 5   status                97 non-null     object 
 6   week                  97 non-null     int64  
 7   season                97 non-null     int64  
 8   FD_Pts_DvP            97 non-null     float64
 9   DK_Pts_DvP            97 non-null     float64
 10  QB_role               97 non-null     float64
 11  completions_L8        97 non-null     float64
 12  attempts_L8           97 non-null     float64
 13  passing_yards_L8      97 non-null     float64
 14  passing_tds_L8        97 non-null     float64
 15  interceptions_L8      97 

In [174]:
qb_df_dk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  67 non-null     object 
 1   position              67 non-null     object 
 2   salary                67 non-null     int64  
 3   team                  67 non-null     object 
 4   opponent              67 non-null     object 
 5   status                67 non-null     object 
 6   week                  67 non-null     int64  
 7   season                67 non-null     int64  
 8   FD_Pts_DvP            67 non-null     float64
 9   DK_Pts_DvP            67 non-null     float64
 10  QB_role               67 non-null     float64
 11  completions_L8        67 non-null     float64
 12  attempts_L8           67 non-null     float64
 13  passing_yards_L8      67 non-null     float64
 14  passing_tds_L8        67 non-null     float64
 15  interceptions_L8      67 

In [175]:
qb_df_fd = qb_df_fd.set_index(['name', 'team', 'position', 'salary', 'opponent', 'status', 'season', 'week'], drop = True)
qb_df_dk = qb_df_dk.set_index(['name', 'team', 'position', 'salary', 'opponent', 'status', 'season', 'week'], drop = True)

In [176]:
qb_df_fd = qb_df_fd[features]
qb_df_dk = qb_df_dk[features]

In [177]:
qb_df_fd[qb_df_fd['QB_role'] == 1]

,,,,,,,,FD_Pts_DvP,DK_Pts_DvP,QB_role,completions_L8,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,...,int_pct,wind,div_game,spread_line,total_line,outdoors,grass,home_team,pred_total,opp_total
name,team,position,salary,opponent,status,season,week,,,,,,,,,,,,,,,,,,,,,
Josh Allen,BUF,QB,9200,TEN,Active,2024,7,15.3300,15.4550,1.0,18.125000,28.125000,193.625,1.750,0.000000,1.250,226.250000,...,0.000000,8,0,8.5,41.5,1,0,1,25.00,16.50
Jayden Daniels,WAS,QB,9100,CAR,Active,2024,7,15.9975,16.5600,1.0,20.833333,27.666667,234.000,1.000,0.333333,2.500,202.166667,...,0.012048,3,0,8.5,51.5,1,1,1,30.00,21.50
Jordan Love,GB,QB,8700,HOU,Active,2024,7,20.3450,21.2200,1.0,21.750000,33.250000,271.125,2.750,1.000000,0.625,282.125000,...,0.030075,9,0,2.5,47.5,1,1,1,25.00,22.50
Jalen Hurts,PHI,QB,8500,NYG,Active,2024,7,16.5525,17.3025,1.0,19.500000,28.875000,208.250,1.250,0.750000,2.375,242.125000,...,0.025974,3,1,3.0,43.5,1,0,0,23.25,20.25
CJ Stroud,HOU,QB,8400,GB,Active,2024,7,17.7300,18.6050,1.0,22.125000,32.750000,253.250,1.625,0.500000,2.000,248.625000,...,0.015267,9,0,-2.5,47.5,1,1,0,22.50,25.00
Joe Burrow,CIN,QB,8200,CLE,Active,2024,7,16.8175,16.8175,1.0,22.750000,32.250000,253.250,1.875,0.500000,2.625,221.500000,...,0.015504,6,1,5.5,41.5,1,1,0,23.50,18.00
Patrick Mahomes,KC,QB,8000,SF,Active,2024,7,17.3650,18.2400,1.0,24.000000,33.500000,253.000,1.375,0.875000,1.750,189.625000,...,0.026119,3,0,-1.5,47.5,1,1,0,23.00,24.50
Jared Goff,DET,QB,7800,MIN,Active,2024,7,18.4475,19.6975,1.0,22.875000,32.500000,270.875,1.500,0.500000,2.000,227.125000,...,0.015385,0,1,-1.5,50.5,0,1,0,24.50,26.00
Brock Purdy,SF,QB,7700,KC,Active,2024,7,18.2625,19.0750,1.0,20.500000,31.750000,268.875,1.375,0.625000,1.875,301.875000,...,0.019685,3,0,1.5,47.5,1,1,1,24.50,23.00


In [178]:
qb_df_dk[qb_df_dk['QB_role'] == 1]

,,,,,,,,FD_Pts_DvP,DK_Pts_DvP,QB_role,completions_L8,attempts_L8,passing_yards_L8,passing_tds_L8,interceptions_L8,sacks_L8,passing_air_yards_L8,...,int_pct,wind,div_game,spread_line,total_line,outdoors,grass,home_team,pred_total,opp_total
name,team,position,salary,opponent,status,season,week,,,,,,,,,,,,,,,,,,,,,
Jayden Daniels,WAS,QB,7600,CAR,Active,2024,7,15.9975,16.5600,1.0,20.833333,27.666667,234.000,1.000,0.333333,2.500,202.166667,...,0.012048,3,0,8.0,52.0,1,1,1,30.00,22.00
Josh Allen,BUF,QB,7500,TEN,Active,2024,7,15.3300,15.4550,1.0,18.125000,28.125000,193.625,1.750,0.000000,1.250,226.250000,...,0.000000,8,0,9.0,41.0,1,0,1,25.00,16.00
Jalen Hurts,PHI,QB,7300,NYG,Active,2024,7,16.5525,17.3025,1.0,19.500000,28.875000,208.250,1.250,0.750000,2.375,242.125000,...,0.025974,3,1,3.0,43.0,1,0,0,23.00,20.00
CJ Stroud,HOU,QB,7100,GB,Active,2024,7,17.7300,18.6050,1.0,22.125000,32.750000,253.250,1.625,0.500000,2.000,248.625000,...,0.015267,9,0,-3.0,47.5,1,1,0,22.25,25.25
Patrick Mahomes,KC,QB,7000,SF,Active,2024,7,17.3650,18.2400,1.0,24.000000,33.500000,253.000,1.375,0.875000,1.750,189.625000,...,0.026119,3,0,-1.5,47.0,1,1,0,22.75,24.25
Jordan Love,GB,QB,6900,HOU,Active,2024,7,20.3450,21.2200,1.0,21.750000,33.250000,271.125,2.750,1.000000,0.625,282.125000,...,0.030075,9,0,3.0,47.5,1,1,1,25.25,22.25
Joe Burrow,CIN,QB,6800,CLE,Active,2024,7,16.8175,16.8175,1.0,22.750000,32.250000,253.250,1.875,0.500000,2.625,221.500000,...,0.015504,6,1,6.0,41.5,1,1,0,23.75,17.75
Jared Goff,DET,QB,6500,MIN,Active,2024,7,18.4475,19.6975,1.0,22.875000,32.500000,270.875,1.500,0.500000,2.000,227.125000,...,0.015385,0,1,-1.5,51.0,0,1,0,24.75,26.25
Brock Purdy,SF,QB,6400,KC,Active,2024,7,18.2625,19.0750,1.0,20.500000,31.750000,268.875,1.375,0.625000,1.875,301.875000,...,0.019685,3,0,1.5,47.0,1,1,1,24.25,22.75


In [179]:
qb_df_fd.to_csv('FD_qb_for_model_' + str(season) + '_' + str(week) + '.csv')
qb_df_dk.to_csv('DK_qb_for_model_' + str(season) + '_' + str(week) + '.csv')

In [504]:
###FANDUEL SCORING
#Rushing yards made = 0.1pts	
#Rushing touchdowns = 6pts	
#Passing yards = 0.04pts	
#Passing touchdowns = 4pts	
#Interceptions = -1pt	
#Receiving yards = 0.1pts	
#Receiving touchdowns = 6pts	
#Receptions = 0.5pts	
#Kickoff return touchdowns = 6pts	
#Punt return touchdowns = 6pts	
#Fumbles lost = -2pts	
#Own fumbles recovered touchdowns = 6pts	
#Two-point conversions scored = 2pts	
#Two-point conversion passes = 2pts	
#Field-goals from 0-39 yards = 3pts	
#Field-goals from 40-49 yards = 4pts	
#Field-goals from 50+ yards = 5pts	
#Extra-point conversions = 1pt

###DRAFTKINGS SCORING
#PAssing TD = 4 pts
#passing yards = .04 pts
#300 passing yards = 3 pts (bonus)
#Interception = -1 pts
#Rushing TD = 6 pts
#Rushing yds = 0.1 pts
#100 yd rushing game = 3 pts (bonus)
#Receiving TD = 6 pts
#Receiving yds = 0.1 pts
#100 receiving yards game = 3 pts (bonus)
#Receptions = 1 pt
#Punt/kickoff/FG return for TD = 6 pts
#Fumble lost = -1 pt
#2 pt conversion (pass, run or catch) = 2 pts
#Offensive fumble recovery TD = 6

In [505]:
# #import sqlite3

# # Establish a connection to the SQLite database
# conn = sqlite3.connect('nfl_dfs.db')

# # Define the table name
# table_name = 'qb_dataset'

# # Specify data types
# dtype = {
#     'season': 'INTEGER',
#     'week': 'INTEGER',
#     'player_id': 'TEXT',
#     'player_display_name': 'TEXT',
#     'team': 'TEXT',
#     'FD_Pts': 'REAL',
#     'DK_Pts': 'REAL',
#     'FD_Pts_DvP': 'REAL',
#     'DK_Pts_DvP': 'REAL',
#     'QB_role': 'REAL',
#     'completions_L8': 'REAL',
#     'attempts_L8': 'REAL',
#     'passing_yards_L8': 'REAL',
#     'passing_tds_L8': 'REAL',
#     'interceptions_L8': 'REAL',
#     'sacks_L8': 'REAL',
#     'passing_air_yards_L8': 'REAL',
#     'pacr_L8': 'REAL',
#     'carries_L8': 'REAL',
#     'rushing_yards_L8': 'REAL',
#     'rushing_tds_L8': 'REAL',
#     'comp_pct': 'REAL',
#     'yds_per_attempt': 'REAL',
#     'td_pct': 'REAL',
#     'sack_pct': 'REAL',
#     'int_pct': 'REAL',
#     'wind': 'REAL',
#     'div_game': 'REAL',
#     'opponent': 'TEXT',
#     'spread_line': 'REAL',
#     'total_line': 'REAL',
#     'outdoors': 'INTEGER',
#     'grass': 'INTEGER',
#     'home_team': 'INTEGER',
#     'pred_total': 'REAL',
#     'opp_total': 'REAL'
# }

# # Write the DataFrame to the SQLite table
# quarterback_df.to_sql(table_name, conn, if_exists='replace', index=False, dtype=dtype)

# # Confirm that the data has been written
# print(f"Data written to table {table_name} in SQLite database nfl_dfs.db")

# # Close the connection
# conn.close()
